# Der Ruin des Spielers


In dieser Fallstudie studieren wir unsere Gewinnchancen beim Spielen im Casino. Insbesondere wollen wir am Ende folgende Frage beantworten können:
*Wenn wir mit einem bestimmten Anfangsbudget ins Casino gehen und unser Ziel ist, einen bestimmten Gewinn zu erzielen. In wie vielen Fällen gelingt uns dies, und in wie vielen Fällen sind wir zuvor ruiniert*.

Dabei werden wir sehen, wie uns Funktionen und Prozeduren helfen eine Aufgabe zu strukturieren. 
Zudem lernen wir auch, wie wir in einer Simulation Zufallszahlen nutzen können. 


### Zufallszahlen generieren

Jede Programmiersprache stellt uns Funktionalität zur Verfügung, um zufällige Zahlen zu generieren. Um zufällige Zahlen in Python zu generieren nutzen wir die Bibliothek ```random```, die wir aber zuerst importieren müssen. 

In [109]:
import random

Mithilfe der Funktion ```random.randint(a, b)``` können wir eine zufällige Ganzahl im Interval $[a, b]$ generieren. 
Das Modul stellt auch viele weitere Funktionen zur Verfügung, die Sie in der [Dokumentation](https://python.readthedocs.io/en/stable/library/random.html) finden. Wir brauchen hier nur diese eine Funktion. 

In [8]:
random.randint(0, 10)

3

Wir sehen, dass wenn wir diese Funktion mehrmals ausführen, dass jedes Mal ein anderer Wert erzeugt wird. Mittels einer Schleife, können wir so viele verschiedene, zufällige Werte generieren. 

In [13]:
i = 0
while i < 10:
    print(random.randint(0, 36))
    i += 1;


7
1
1
6
7
1
9
10
10
0


### Ein Spiel simulieren

Nun nutzen wir diese Zufallsfunktion um ein Roulettespiel zu simulieren. Wir nutzen dazu eine Funktion. Diese bekommt als Argument unseren Einsatz, und gibt den Gewinn oder Verlust zurück. 
Da es keinen Unterschied macht, ob wir auf Rot oder Schwarz setzen (die Gewinnchancen sind immer gleich), bilden wir dies in unserer Funktion nicht ab. Wir implementieren nur den Sachverhalt, dass wir in 19 von 37 Fällen verlieren und in 18 gewinnen. 

In [110]:
def play_game(bet): 
    random_number = random.randint(0, 36)
    if random_number == 0:
        return -bet
    elif random_number <= 18:
        return -bet
    else: 
        return bet

In [111]:
play_game(5)

5

### Der Durchschnittliche Gewinn

Die vielleicht einfachste interessante Frage wäre, wieviel Geld wir im Durchschnitt gewinnen. Bei diesem einfachen Spiel könnten wir dies noch ohne Programm überprüfen. Bei komplexeren Spielen, wie wir sie später ansehen, brauchen wir aber solche Simulationen. 

In [113]:
def average_of_n_games(n, bet):
    cumulated_win = 0
    i = 0
    while i < n:
        cumulated_win = cumulated_win + play_game(bet)
        i = i + 1
    return cumulated_win / n

Wir können nun überprüfen, dass wenn ```n``` gross wird, wir immer einen Verlust einfahren werden. Dies ist der Gewinn des Casinos. Je grösser die Anzahl Spieler, desto stabiler der Gewinn. 

In [119]:
average_of_n_games(10000, 100)

-3.8

### Spielen mit Zielbetrag

Statt auf ein einziges Spiel, fokussieren wir uns nun auf eine Serie von Spielen. Wir spielen so lange weiter, bis wir einen bestimmten Zielbetrag erreicht haben. Wenn wir aber kein Geld mehr haben, dann müssen wir aufhören zu spielen. Dieses Szenario wird im folgenden Spiel definiert. Wir beachten, dass wir die Funktion ```play_game``` hier benutzen. Wir nutzen diese Funktion als *black box*, d.h. wir müssen gar nicht wissen, welches Spiel da eigentlich simuliert wird. Wichtig ist nur, dass wir wissen, wie wir die Funktion aufrufen und das der Rückgabewert jeweils unser Gewinn oder Verlust ist. 

In [94]:
def play_until_target_reached_or_ruined(initial_amount, bet_per_game, target_amount):
    money_to_play_with = initial_amount

    while money_to_play_with >= bet_per_game and money_to_play_with < target_amount:
        money_to_play_with += play_game(bet_per_game)

    return money_to_play_with >= target_amount

In [95]:
play_until_target_reached_or_ruined(100, 10, 300)

False

### Simulation - Wie viele Spiele gewinnen wir im Schnitt

Zum Schluss schreiben wir uns eine Funktion, die uns eine Anzahl Spiele simuliert und dabei zählt, wie oft wir gewinnen. Auch hier nurzen wir nun wieder die zuvor definierte Funktion. 

In [107]:
def count_wins_in_n_games(number_of_tries, initial_amount, bet_per_game, target_amount):

    num_wins = 0

    i = 0
    while i < number_of_tries:
        if play_until_target_reached_or_ruined(initial_amount, bet_per_game, target_amount) == True:
            num_wins = num_wins + 1
        i += 1

    return num_wins

Nun können wir in der Simulation austesten, wie erfolgsversprechend unsere Strategie ist. 

In [108]:
n_games = 1000

number_of_wins = count_wins_in_n_games(n_games, 1000, 500, 1500)
print("number of wins: ", number_of_wins)
print("number of ruins: ", n_games - number_of_wins)

number of wins:  656
number of ruins:  344
